In [3]:
import pandas as pd 
import json

In [4]:
my_json_url = 'https://jackshutler.github.io/datasets/CABLES_QA.json'
cables_1000 = "https://jackshutler.github.io/datasets/cables_1000.parquet"


cables = pd.read_json(my_json_url)

with open("cables_qa_keyword.json", "r") as json_file:
    cables_keyword = json.load(json_file)

cables_1000 = pd.read_parquet(cables_1000)

cables_1000

,id,key,subject,sender,recipient,date,msg,tags,concepts,class
791058,1250029,odbc://SAS_1970s/dbo.SAS_Docs/1151258,n/a,STATE,KUALA LUMPUR,09 Aug 1975,PAGE 01 STATE 189179\n\n51 \nORIGIN EA-1...,"PDIP, MY, US, (KISSINGER, HENRY A), (TENGKU AH...",CONGRATULATORY MESSAGES,UNCLASSIFIED
922999,1427467,odbc://SAS_1970s/dbo.SAS_Docs/898204,TRAIN TRANSIT OF GDR,BERLIN,MOSCOW,20 Sep 1975,LIMITED OFFICIAL USE\n\nPAGE 01 BERLIN 06413 ...,"PFOR, OTRA, WB, GC","BERLIN RAILROAD ACCESS, TRAVEL CONTROLS",UNCLASSIFIED
723920,1138837,odbc://SAS_1970s/dbo.SAS_Docs/2333802,CODEL DUPONT,JIDDA,STATE,12 May 1975,LIMITED OFFICIAL USE\n\nPAGE 01 JIDDA 03323 ...,"OREP, (DUPONT, PIERRE S)",n/a,UNCLASSIFIED
369278,653182,odbc://SAS_1970s/dbo.SAS_Docs/2869531,ICAO SECRETARY GENERAL ELECTION,PORT OF SPAIN,STATE,01 Apr 1976,LIMITED OFFICIAL USE POSS DUPE\n\nPAGE 01 ...,"EAIR, PORG, TD, US, ICAO","SECRETARY GENERAL, INSTRUCTIONS, NOMINATION, P...",UNCLASSIFIED
297501,576044,odbc://SAS_1970s/dbo.SAS_Docs/2647682,SURPLUS PROPERTY,STATE,NEW DELHI,24 Apr 1974,LIMITED OFFICIAL USE\n\nPAGE 01 STATE 083687...,"EFIN, IN","MEETING AGENDA, SURPLUS PROPERTY SALES, NEGOTI...",UNCLASSIFIED
...,...,...,...,...,...,...,...,...,...,...
877097,1351393,odbc://SAS_1970s/dbo.SAS_Docs/3153468,EXCHANGES - 1974/75 US LECTURER PROGRAM - ARRI...,STATE,MOSCOW,20 Feb 1975,UNCLASSIFIED\n\nPAGE 01 STATE 038445\n\n62\n...,"OEXC, UR, (LU, LE WU)",n/a,UNCLASSIFIED
1089384,1861407,odbc://SAS/SAS.dbo.SAS_Docs/e539122c-c288-dd11...,OPIC\'S COUNSEL WISS TDY,JAKARTA,STATE,19 Oct 1977,UNCLASSIFIED\n\nPAGE 01 JAKART 14165 1...,"EINV, OTRA, ID, OPIC","LITIGATION, PERSONNEL TRAVEL, VISITS",UNCLASSIFIED
1276494,2144042,odbc://SAS/SAS.dbo.SAS_Docs/9895ff38-c288-dd11...,RHODESIANS ADMIT AIR ATTACKS ON ZAMBIA,NAIROBI,STATE,02 Sep 1977,CONFIDENTIAL\n\nPAGE 01 NAIROB 11328 0...,"PDEV, RH, ZA","AIR COMBAT OPERATIONS, GOVERNMENT REACTIONS",UNCLASSIFIED
143119,399564,odbc://SAS_1970s/dbo.SAS_Docs/1345946,EL SALVADOR CASP; IAGS PROGRAM,SECSTATE WASHDC,SAN SALVADOR,16 Apr 1973,CONFIDENTIAL\n\n PAGE 01 STATE 070978\n\n66...,"MARR, PFOR, ES, US",,UNCLASSIFIED


In [5]:
paragraphs = [paragraph for key, paragraph in cables_keyword["data"].items()]
cables_keyword["data"] = paragraphs

In [6]:
cables_json = {"data": list(cables.data)}
contexts = {doc["paragraphs"][0]['document_id']: doc["paragraphs"][0]['context'] for doc in cables_json["data"]}

for i, row in cables_1000.iterrows():
    contexts[str(row["id"])] = row["msg"]

len(contexts)

1055

In [35]:
from collections import defaultdict
def countOccurrences(context, word):
     
    # split the string by spaces in a
    a = context.split(" ")
 
    # search for pattern in a
    count = 0
    for i in range(0, len(a)):
         
        # if match found increase count
        if (word == a[i]):
           count = count + 1
            
    return count      
 
# Driver code
str ="GeeksforGeeks A computer science portal for geeks  "
word ="portal"
print(countOccurrences(str, word))

def keyword_search(contexts, query, preprocessing=True):
    if preprocessing:
        query = query.upper()
        query = query.split()
    else:
        query = [query]
    
    matching_contexts = defaultdict(int)
    for doc_id, context in contexts.items():
        for word in query:
            matching_contexts[doc_id] += countOccurrences(context, word) #.append(word)
    # return the doc_ids sorted by the number of found words in a given context
    return list(dict(sorted(matching_contexts.items(), key=lambda ele: ele[1], reverse=True)).keys())



1


In [36]:

from pprint import pprint

def test_keyword_search(squad_json, contexts, top_k=5, preprocessing=False):
    hits = 0
    num_questions = 0
    for doc in squad_json["data"]:
        # print(type(doc))
        # print(doc)
        paragraphs = doc["paragraphs"][0]
        qas = paragraphs["qas"]
        context = paragraphs["context"]
        document_id = paragraphs["document_id"]
        for qa in qas:
            possible_matches = keyword_search(contexts, qa["question"], preprocessing)    
            # print(possible_matches)

            # print(len(possible_matches))
            # search through the top_k matches, or however many we're found
            # print(document_id)
            # print(possible_matches[:min(len(possible_matches), top_k)])
            hits += 1 if document_id in possible_matches[:min(len(possible_matches), top_k)] else 0
            num_questions += 1
    
    return hits / num_questions

print(test_keyword_search(cables_json, contexts, 5, True))
# print(cables_keyword["data"])
print(test_keyword_search(cables_keyword, contexts, 5, True))
    

0.0
0.5648148148148148


In [ ]:
len(set([i for i, context in contexts.items()]))

1055